# Compare rand behavior
* Different python versions
* Different seeding methods
* MATLAB / Octave 

In [1]:
import matlab.engine
from full_model import *
from scipy.io import loadmat
data_loaded = loadmat("./saved/processed/yaleB.mat")
labels = data_loaded['Y'].reshape(-1)
grps = eng.SSC_modified(37, 0, False, 20.0, False, 1, 1e-20, 16, True, 0)
labels_pred = np.asarray(grps, dtype=np.int32).flatten()
print(1-err_rate(labels, labels_pred), nmi(labels, labels_pred, average_method="geometric"), ari(labels, labels_pred))


Starting MATLAB engine...
-------------------------
Elapsed: 3.42 sec
0.9617021276595745 0.9851113230275155 0.9578386188385504


In [6]:
import matlab.engine
import full_model
from scipy.io import loadmat
data_loaded = loadmat("./saved/processed/yaleB.mat")
images_norm = data_loaded['X']
labels = data_loaded['Y'].reshape(-1)
full_model.run_model(images_norm, labels, seed=0)


Finding affinity matrix (iter: 6)...
-------------------------------------
Elapsed: 4.07 sec

Training Autoencoder...
-----------------------
Layer 1
300 -> 200
epoch 0: global loss = 1.0553826093673706
epoch 100: global loss = 0.033024195581674576
Layer 2
200 -> 150
epoch 0: global loss = 1.2791131734848022
epoch 100: global loss = 0.02575872465968132

epoch 0: global loss = 0.034736163914203644
epoch 25: global loss = 0.002287367358803749
epoch 50: global loss = 0.00209440547041595
epoch 75: global loss = 0.002039118204265833
epoch 100: global loss = 0.002007222268730402
Elapsed: 38.06 sec

Clustering with SSC (iter: 16)...
---------------------------------
Elapsed: 14.90 sec



(0.9625531914893617, 0.9853149048310011, 0.9583452282183348)

# Check simpler models
* Plain SSC
* Plain AE

In [1]:
import matlab.engine
import full_model
from scipy.io import loadmat
data_loaded = loadmat("./saved/processed/yaleB.mat")
images_norm = data_loaded['X']
labels = data_loaded['Y'].reshape(-1)
full_model.run_ae(images_norm, labels, seed=0)


Starting MATLAB engine...
-------------------------
Elapsed: 30.63 sec

Training Autoencoder...
-----------------------
Layer 1
300 -> 200
epoch 0: global loss = 1.0553826093673706
epoch 100: global loss = 0.033024195581674576
Layer 2
200 -> 150
epoch 0: global loss = 1.2791131734848022
epoch 100: global loss = 0.02575872465968132

epoch 0: global loss = 0.03473581373691559
epoch 25: global loss = 0.0022870430257171392
epoch 50: global loss = 0.002094089752063155
epoch 75: global loss = 0.002038809238001704
epoch 100: global loss = 0.0020069198217242956
Elapsed: 137.40 sec

Clustering with SSC (iter: 16)...
---------------------------------
Elapsed: 113.07 sec



(0.9617021276595745, 0.9851113230275155, 0.9578386188385504)

In [ ]:
import matlab.engine
from optimize import *
optimize(gp_minimize, get_params(7), 13, random_seed=0)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 311.7222
Function value obtained: 0.5345
Current minimum: 0.5345
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 246.1591
Function value obtained: 0.7409
Current minimum: 0.5345
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 286.9545
Function value obtained: 0.0404
Current minimum: 0.0404
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 282.9618
Function value obtained: 0.1953
Current minimum: 0.0404
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 296.4713
Function value obtained: 0.9723
Current minimum: 0.0404
Iteration No: 6 started. 

# Reload skopt optimizations
The package *skopt* (scikit-optimize) contains several useful methods that optimize expensive and noisy functions

In [1]:
from copy import deepcopy
from skopt.optimizer import base_minimize

def reload_bad(result, addtl_calls):
    args = deepcopy(result.specs['args'])
    args['n_calls'] = addtl_calls
    args['n_random_starts'] = 0
    args['x0'] = deepcopy(result.x_iters)
    args['y0'] = deepcopy(result.func_vals)
    args['random_state'] = deepcopy(result.random_state)
    return base_minimize(**args)

In [21]:
import numpy as np
import warnings
from copy import deepcopy
from skopt.learning import GaussianProcessRegressor
from skopt.optimizer import base_minimize
from sklearn.utils import check_random_state

def func_new(params):
    global func_, xs_, ys_
    if(len(xs_) > 0):
        y = ys_.pop(0)
        if(params != xs_.pop(0)):
            warnings.warn("Deviated from expected value, re-evaluating", RuntimeWarning)
        else:
            return y
    return func_(params)

def reload(result, addtl_calls, init_seed):
    args = deepcopy(result.specs['args'])
    args['n_calls'] += addtl_calls
    
    # global b/c I couldn't find a better way to pass
    global func_, xs_, ys_
    func_ = args['func']
    xs_ = list(result.x_iters)
    ys_ = list(result.func_vals)
    args['func'] = func_new
    
    # recover initial random_state
    if(isinstance(args['random_state'], np.random.RandomState)):
        args['random_state'] = check_random_state(init_seed)
        # if gp_minimize
        if(isinstance(result.specs['args']['base_estimator'], GaussianProcessRegressor)):
            args['random_state'].randint(0, np.iinfo(np.int32).max)
    
    # run the optimization
    result_new = base_minimize(**args)
    
    # change the function back, to reload multiple times
    result_new.specs['args']['func'] = func_
    
    return result_new

In [60]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

space = [Real(-2, 2, "uniform", name='x'),
         Real(-1, 3, "uniform", name='y')]

@use_named_args(space)
def rosenbrock(x, y):
    # true minimum is 0 at (a, a^2)
    a = 1
    b = 100
    return (a-x)**2 + b*(y-x*x)**2

result1 = gp_minimize(rosenbrock, space, n_calls=20, random_state=0)
result1 = reload(result1, addtl_calls=20, init_seed=0)

result2 = gp_minimize(rosenbrock, space, n_calls=40, random_state=0)

if(np.allclose(result1.func_vals, result2.func_vals)):
    print("TEST PASSED")
else:
    print("TEST FAILED")
print("OPTIMUM:", result1.fun, "at", result1.x)

TEST PASSED
OPTIMUM: 0.17739012894583892 at [0.5878821579502587, 0.35429393738241166]
